<a href="https://colab.research.google.com/github/FG2511/MLP_ForFoodPreparation/blob/master/model1_cross_validation_risultati_Federica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math
import numpy as np
from keras import optimizers
from sklearn.utils import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

import sys
sys.path.append('local_modules')

import mlp
import postprocessing_sw
import cooking_inst_mod
import utils

#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
#@title CHOOSE

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []

true_p =[]
true_n = []
precision = []
fscore= []

test_time =[]
real = []

dimSplit = math.floor(len(dataset[:,0])/10)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  model = mlp.generate_model_leaky(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epochs = 9 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epochs, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #CONFUSION MATRIX AND METRICS FOR MEAN EVALUATION
  tn, fp, fn, tp = confusion_matrix(Y[test], flat_pred).ravel()
  tnr = 100*tn/(tn+fp)
  tpr = 100*tp/(fn+tp)
  prec = 100*(tp/(tp+fp))
  F1 = 2*((prec*tpr)/(prec+tpr))
  
  true_p.append(tpr)
  true_n.append(tnr)
  precision.append(prec)
  fscore.append(F1)
  
  i+=1

print("\n\n")
print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))
print("MEAN PRECISION: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("MEAN F1 SCORE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(fscore), np.std(fscore)))

#FLATTENING LISTS
flat_predictions = [item for sublist in predictions for item in sublist]
true_y = [arr.tolist() for arr in real]
flat_true_y = [item for sublist in true_y for item in sublist]
flat_time = [item for sublist in test_time for item in sublist]


#CONFUSION MATRIX AND METRICS BEFORE POST PROCESSING
print("\n\nCompute confusion matrix and metrics BEFORE POST PROCESSING...")
utils.compute_metrics(flat_true_y, flat_predictions)


Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE
Instructions for updating:
Colocations handled automatically by placer.

FOLD: 1

Generate model...

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/9
315441/315441 [==============================] - 19s 60us/step - loss: 0.5212 - acc: 0.7733
Epoch 2/9
315441/315441 [==============================] - 18s 57us/step - loss: 0.4154 - acc: 0.8366
Epoch 3/9
315441/315441 [==============================] - 18s 56us/step - loss: 0.3910 - acc: 0.8335
Epoch 4/9
315441/315441 [==============================] - 17s 53us/step - loss: 0.3822 - acc: 0.8300
Epoch 5/9
315441/315441 [==============================] - 18s 57us/step - loss: 0.3755 - acc: 0.8295
Epoch 6/9
315441/315441 [==============================] - 17s 55us/step - loss: 0.3689 - acc: 0.8295
Epoch 7/9
315441/315441 

In [3]:
#CORRECTIONS: SETTING INSTANCES BETWEEN 01:00 am AND 05:00 am TO 0

n_pred = []

for p,t in zip(flat_predictions, flat_time):

  if(t>59.0 and t<300.0):
     n_pred.append(0)
  else:
    n_pred.append(p)

print("\n\nCompute confusion matrix and metrics AFTER CORRECTIONS...")
utils.compute_metrics(flat_true_y, n_pred)



Compute confusion matrix and metrics AFTER CORRECTIONS...
TN 290134
FP 44033
FN 3580
TP 12744
ACCURACY: 86.42 %
TRUE NEGATIVE RATE (SPECIFICITY): 86.82 %
TRUE POSITIVE RATE (RECALL): 78.07 %
PRECISION: 22.45 %
F1 SCORE: 34.87 %


In [4]:
#POST PROCESSING WITH SLIDING WINDOWS (MINUTE BY MINUTE)
new_pred =  postprocessing_sw.sliding_windows(n_pred,35)
new_pred_1 = postprocessing_sw.sliding_windows(flat_predictions,35)

#CONFUSION MATRIX AND METRICS
print("\n\nCompute confusion matrix and metrics AFTER POST PROCESSING ONLY...")
utils.compute_metrics(flat_true_y, new_pred_1)
print("\n\nCompute confusion matrix and metrics AFTER CORRECTIONS AND POST PROCESSING...")
utils.compute_metrics(flat_true_y, new_pred)



SLIDING WINDOWS FUNCTION...

SLIDING WINDOWS FUNCTION...


Compute confusion matrix and metrics AFTER POST PROCESSING ONLY...
TN 293863
FP 40304
FN 3815
TP 12509
ACCURACY: 87.41 %
TRUE NEGATIVE RATE (SPECIFICITY): 87.94 %
TRUE POSITIVE RATE (RECALL): 76.63 %
PRECISION: 23.69 %
F1 SCORE: 36.19 %


Compute confusion matrix and metrics AFTER CORRECTIONS AND POST PROCESSING...
TN 293917
FP 40250
FN 3815
TP 12509
ACCURACY: 87.43 %
TRUE NEGATIVE RATE (SPECIFICITY): 87.96 %
TRUE POSITIVE RATE (RECALL): 76.63 %
PRECISION: 23.71 %
F1 SCORE: 36.21 %


In [5]:
#COOKING INSTANCE MODALITY

#before post-processing
print("\nCOOKING INSTANCE MODALITY BEFORE POST PROCESSING")
cooking_inst_mod.get_precision_recall_f1(flat_predictions,flat_true_y)
#after corrections only
print("\nCOOKING INSTANCE MODALITY AFTER CORRECTIONS ONLY")
cooking_inst_mod.get_precision_recall_f1(n_pred,flat_true_y)
#after post-processing only
print("\nCOOKING INSTANCE MODALITY AFTER POST PROCESSING ONLY ")
cooking_inst_mod.get_precision_recall_f1(new_pred_1,flat_true_y)
#after post-processing and corrections
print("\nCOOKING INSTANCE MODALITY AFTER CORRECTIONS AND POST PROCESSING")
cooking_inst_mod.get_precision_recall_f1(new_pred,flat_true_y)


COOKING INSTANCE MODALITY BEFORE POST PROCESSING
N° pasti reali: 669
N° pasti predetti: 1567
TP: 558
FP: 1009
FN: 111
Recall: 83.41 %
Precision: 35.61 %
F1 score: 49.91 %

COOKING INSTANCE MODALITY AFTER CORRECTIONS ONLY
N° pasti reali: 669
N° pasti predetti: 1561
TP: 558
FP: 1003
FN: 111
Recall: 83.41 %
Precision: 35.75 %
F1 score: 50.04 %

COOKING INSTANCE MODALITY AFTER POST PROCESSING ONLY 
N° pasti reali: 669
N° pasti predetti: 902
TP: 514
FP: 461
FN: 155
Recall: 76.83 %
Precision: 52.72 %
F1 score: 62.53 %

COOKING INSTANCE MODALITY AFTER CORRECTIONS AND POST PROCESSING
N° pasti reali: 669
N° pasti predetti: 900
TP: 514
FP: 459
FN: 155
Recall: 76.83 %
Precision: 52.83 %
F1 score: 62.61 %
